# Testing Avoid Repetion when Engine have a better position

In [23]:
using Pkg
# Pkg.add("NBInclude")
using NBInclude
# Pkg.add("Chess")
using Chess

In [24]:
@nbinclude("Play.ipynb")

In [25]:
# Define a set about the moves (a8,b8,b7)
# 1. Loop above game is over (checkmate or draw)
# 2. Make Schnittmenge from moves to possible moves
# 3. Move First Move from Schnittmenge
# 3.1 If no Schnittmenge, give User input
# 4. Calculate with Iterative Deepening the best move for black and move it.
# 5. Repeat 1-4 until game is over
function testAvoidRepForWhite(game)
    preferredMoves = Set([movefromstring("b8a8"), movefromstring("a8b8"), movefromstring("a8b7"), movefromstring("b7a8"), movefromstring("b7b8"),movefromstring("b8b7") ])
    aBoard = AdvBoard(deepcopy(board(game)))
    cache = initCache()
    while !isterminal(game)
        printGame(game)
        possibleMoves = Set(moves(board(game)))
        intersection = intersect(preferredMoves, possibleMoves)
        if length(intersection) > 0
            move = first(intersection)
        else
            println("Make a move.")
            userInput = readline()
            move = movefromstring(userInput)
        end
        clearCache(aBoard, move, cache)
        cache = initCache()
        #println("Cache: ", aBoard.repCounter)
        #println("Board: ", aBoard.state)
        #println("zobrist hash: ", zobrist_hash(aBoard.state))
        #println("amount: ", aBoard.repCounter[zobrist_hash(aBoard.state)])
        domove!(game, move)
        domoveAdv!(aBoard, move)
        print(aBoard.repCounter)
        playMove(game, aBoard, cache)
    end
    println("--------------------------------------")
    println("Cache: ", aBoard.repCounter)
    println("Board: ", aBoard.state)
    println("zobrist hash: ", zobrist_hash(aBoard.state))
    println("amount: ", aBoard.repCounter[zobrist_hash(aBoard.state)])
    setWin(game, "")
    println(game)
    saveGameToPGN(game)
end

testAvoidRepForWhite (generic function with 1 method)

In [26]:
b1 = fromfen("K7/P6p/2q5/3p1k2/8/8/8/8 w - - 0 1")
testAvoidRepForWhite(Game(b1))

Board (K7/P6p/2q5/3p1k2/8/8/8/8 w - -):
 K  -  -  -  -  -  -  - 
 P  -  -  -  -  -  -  p 
 -  -  q  -  -  -  -  - 
 -  -  -  p  -  k  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  -

Did not clear cache
Dict{UInt64, Int8}(0xfe67a2ab9b1dc091 => 1, 0x5a1826eda554dac0 => 1)The best value was calculated with a depth of 1 and it took 0.0 seconds.
The best value was calculated with a depth of 2 and it took 0.0 seconds.
The best value was calculated with a depth of 3 and it took 0.003999948501586914 seconds.
The best value was calculated with a depth of 4 and it took 0.006999969482421875 seconds.
The best value was calculated with a depth of 5 and it took 0.037000179290771484 seconds.
The best value was calculated with a depth of 6 and it took 0.2239999771118164 seconds.
The best value was calculated with a depth of 7 and it took 0.4760000705718994 seconds.
The best value was calculated with a depth of 8 and it took 1.6639997959136963 seconds.
-1165
Move[Move(c6b6)]The Engine needs to be calculated the bestmove: 9.91700005531311 seconds
Current board score: -1050
Evaluation of engine: -1165
Current entries in Cache: 636321
Computer play: Move(c6b6)
Did not clear cache


Board (1K6/P6p/1q6/3p1k2/8/8/8/8 w - -):
 -  K  -  -  -  -  -  - 
 P  -  -  -  -  -  -  p 
 -  q  -  -  -  -  -  - 
 -  -  -  p  -  k  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  -

Did not clear cache
Dict{UInt64, Int8}(0xfe67a2ab9b1dc091 => 1, 0xeb499d0d68188096 => 1, 0x4f36194b56519ac7 => 1, 0x5a1826eda554dac0 => 1)The best value was calculated with a depth of 1 and it took 0.0 seconds.
The best value was calculated with a depth of 2 and it took 0.0009999275207519531 seconds.
The best value was calculated with a depth of 3 and it took 0.002000093460083008 seconds.
The best value was calculated with a depth of 4 and it took 0.0029997825622558594 seconds.
The best value was calculated with a depth of 5 and it took 0.026999950408935547 seconds.
The best value was calculated with a depth of 6 and it took 0.10600018501281738 seconds.
The best value was calculated with a depth of 7 and it took 0.26399993896484375 seconds.
The best value was calculated with a depth of 8 and it took 0.5559999942779541 seconds.
The best value was calculated with a depth of 9 and it took 1.4270000457763672 seconds.
-1040
Move[Move(b6a6), Move(b6a5), Move(b6c6), Move(b6c5), Move(b6d8), Mo

Board (K7/P6p/q7/3p1k2/8/8/8/8 w - -):
 K  -  -  -  -  -  -  - 
 P  -  -  -  -  -  -  p 
 q  -  -  -  -  -  -  - 
 -  -  -  p  -  k  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  -

...

Did not clear cache
Dict{UInt64, Int8}(0x7f7525167af034b6 => 2, 0x92e8b5f5bf002c5d => 1, 0xc9e5364bf8a8b5ac => 1, 0x9261dfd09f48e962 => 1, 0xe3f1dcb66413178f => 1, 0x1871bda97fe2fdf9 => 1, 0xf146440883ea36a3 => 1, 0x4d5024e87b813d12 => 1, 0xfe67a2ab9b1dc091 => 1, 0xeb499d0d68188096 => 2, 0xab85b19c3d56ac45 => 1, 0xf613e264fe383362 => 2, 0x1b2d68917a4a016d => 2, 0x9efcd11072f31bbb => 2, 0x9475f44afb8daba3 => 2, 0xc22736fbf71bc454 => 1, 0x7afb39ed898de65a => 2, 0x24f1cc8d1d106878 => 1, 0x249f64bc6f4b9488 => 1, 0xf04028bed5fa1d09 => 1, 0x9e43507c1a2641de => 2, 0x7d045453defaccb4 => 2, 0x2d7035164a73ad57 => 2, 0x21bb3c6e0175a1ea => 2, 0xbf52ecd744031b3c => 1, 0x34cdfee29d609feb => 1, 0xa2f70b3152cc08c9 => 1, 0x7e31561b0f70cfe5 => 1, 0x333a2a0810c418dd => 1, 0xc39eecb37ab598bd => 1, 0x9d45071c47cd9d2c => 1, 0xad22228f6ed7b329 => 2, 0x9745ae4e2e8d028c => 2, 0x23529c713ab7ddec => 1, 0xc3215a4da10beffe => 1, 0xf00158075ab6f08b => 1, 0x4f36194b56519ac7 => 2, 0x40a9810a1daa5da3 => 1, 0x4925fde02

Board (K7/P6p/3q4/5k2/8/8/3p4/8 w - -):
 K  -  -  -  -  -  -  - 
 P  -  -  -  -  -  -  p 
 -  -  -  q  -  -  -  - 
 -  -  -  -  -  k  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  p  -  -  -  - 
 -  -  -  -  -  -  -  -

Did not clear cache
Dict{UInt64, Int8}(0x7f7525167af034b6 => 2, 0x92e8b5f5bf002c5d => 1, 0xc9e5364bf8a8b5ac => 1, 0x9261dfd09f48e962 => 1, 0xe3f1dcb66413178f => 1, 0x1871bda97fe2fdf9 => 1, 0x7ff4300b54db0bff => 1, 0xf146440883ea36a3 => 1, 0x4d5024e87b813d12 => 1, 0xfe67a2ab9b1dc091 => 1, 0xeb499d0d68188096 => 2, 0xab85b19c3d56ac45 => 1, 0xf613e264fe383362 => 2, 0x1b2d68917a4a016d => 2, 0x9efcd11072f31bbb => 2, 0x9475f44afb8daba3 => 2, 0xc22736fbf71bc454 => 1, 0x7afb39ed898de65a => 2, 0x24f1cc8d1d106878 => 1, 0x249f64bc6f4b9488 => 1, 0xf04028bed5fa1d09 => 1, 0x9e43507c1a2641de => 2, 0x7d045453defaccb4 => 2, 0x2d7035164a73ad57 => 2, 0x21bb3c6e0175a1ea => 2, 0x2470d990333b5731 => 1, 0xbf52ecd744031b3c => 1, 0x34cdfee29d609feb => 1, 0xa2f70b3152cc08c9 => 1, 0x7e31561b0f70cfe5 => 1, 0x333a2a0810c418dd => 1, 0xc39eecb37ab598bd => 1, 0x9d45071c47cd9d2c => 1, 0xad22228f6ed7b329 => 2, 0x9745ae4e2e8d028c => 2, 0x23529c713ab7ddec => 1, 0xc3215a4da10beffe => 1, 0xf00158075ab6f08b => 1, 0x4f36194b5

Board (8/PK5p/3q4/5k2/8/8/8/3q4 w - -):
 -  -  -  -  -  -  -  - 
 P  K  -  -  -  -  -  p 
 -  -  -  q  -  -  -  - 
 -  -  -  -  -  k  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  q  -  -  -  -

Did not clear cache
Dict{UInt64, Int8}(0x7f7525167af034b6 => 2, 0x92e8b5f5bf002c5d => 1, 0xc9e5364bf8a8b5ac => 1, 0x9261dfd09f48e962 => 1, 0xe3f1dcb66413178f => 1, 0x1871bda97fe2fdf9 => 1, 0x7ff4300b54db0bff => 1, 0xf146440883ea36a3 => 1, 0x4d5024e87b813d12 => 1, 0xfe67a2ab9b1dc091 => 1, 0xeb499d0d68188096 => 2, 0xab85b19c3d56ac45 => 1, 0xf613e264fe383362 => 2, 0x1b2d68917a4a016d => 2, 0x9efcd11072f31bbb => 2, 0x9475f44afb8daba3 => 2, 0xc22736fbf71bc454 => 1, 0x7afb39ed898de65a => 2, 0x24f1cc8d1d106878 => 1, 0x249f64bc6f4b9488 => 1, 0xf04028bed5fa1d09 => 1, 0x9e43507c1a2641de => 2, 0x7d045453defaccb4 => 2, 0x2d7035164a73ad57 => 2, 0x21bb3c6e0175a1ea => 2, 0x2470d990333b5731 => 1, 0xbf52ecd744031b3c => 1, 0x34cdfee29d609feb => 1, 0xa2f70b3152cc08c9 => 1, 0x7e31561b0f70cfe5 => 1, 0x333a2a0810c418dd => 1, 0xc39eecb37ab598bd => 1, 0x9d45071c47cd9d2c => 1, 0xad22228f6ed7b329 => 2, 0x9745ae4e2e8d028c => 2, 0x23529c713ab7ddec => 1, 0xc3215a4da10beffe => 1, 0xf00158075ab6f08b => 1, 0x2a89e6f47

In [15]:
a = AdvBoard(fromfen("8/PK5p/8/3p1k2/3q4/8/8/8 b - - 1 1"))

d = Dict{UInt64, Int8}(thishash => 2)
# a.repCounter = Dict{UInt64, Int8}(0x47b34b3654677144 => 2, 0xc1bfb250bdd6dc7a => 1, 0x2bb36ca1dc9e9ed6 => 2, 0x2473feb2579c1f57 => 2, 0x42d45b72cc4520a9 => 1, 0xc4d8a21425f48d97 => 2, 0xf8a9fa4a73eb68c2 => 2, 0xf50aaa48ecfe9f07 => 2, 0xf7696859f8e9e943 => 2, 0x7661436a9d6d63d4 => 2)
a.repCounter = Dict{UInt64, Int8}(zobrist_hash(fromfen("8/PK5p/8/3p1k2/1q6/8/8/8 w - - 1 1")) => 2)

print(iterativeDeepening(a, 8, initCache(), true, 1.0, true))

The best value was calculated with a depth of 1 and it took 0.0009999275207519531 seconds.
The best value was calculated with a depth of 2 and it took 0.002000093460083008 seconds.
The best value was calculated with a depth of 3 and it took 0.009999990463256836 seconds.
The best value was calculated with a depth of 4 and it took 0.021000146865844727 seconds.
The best value was calculated with a depth of 5 and it took 0.08599996566772461 seconds.
The best value was calculated with a depth of 6 and it took 0.10400009155273438 seconds.
The best value was calculated with a depth of 7 and it took 1.0049998760223389 seconds.
The best value was calculated with a depth of 8 and it took 1.9750001430511475 seconds.
-1115
Move[Move(d4b2)](-1115, Move(d4b2))